In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime, time
import sklearn, sklearn.metrics, sklearn.preprocessing, sklearn.linear_model, sklearn.ensemble, sklearn.model_selection
import nltk, nltk.stem
import pickle
import collections

import modutils

data_dir = '../DataSets/MercariPrice/'
src_file = data_dir + 'train_title.csv' 
stat_file = '21MerPrice04_TextStat.csv'
w2vsrc_file = '21MerPrice04_W2VSrc.csv'
meta_file = 'Models-21MerPrice04-W2V/model-24.ckpt.meta'

In [2]:
#Load state

with open(stat_file, 'rb') as f:
    statNwords = pickle.load(f)

with open(w2vsrc_file, 'rb') as f:
    w2v_src = pickle.load(f)
    
mapper = {x[0]:i for (i,x) in enumerate(statNwords)}

def word2idx(w):
    if w in mapper:
        return mapper[w]+1
    else:
        return 0
    
def idx2word(i):
    if i == 0:
        return '<unk>'
    if i-1 >= len(statNwords):
        return '<err>'
    return statNwords[i-1][0]

In [3]:
DICT_SIZE = len(statNwords) + 1
EMBED_SIZE = 50
NCE_NUM_SAMPLED = 50

tf.reset_default_graph()

with tf.name_scope('Input'):
    tf_in_word = tf.placeholder(tf.int32, shape=(None, ))
    tf_in_context = tf.placeholder(tf.int32, shape=(None, 1))
    
with tf.name_scope('Embedding'):
    tf_embedding = tf.Variable(np.random.normal(size=(DICT_SIZE, EMBED_SIZE)) / np.sqrt(EMBED_SIZE), dtype=tf.float32)
    tf_embedded_word = tf.nn.embedding_lookup(tf_embedding, tf_in_word)
    
with tf.name_scope('Training'):
    tf_nce_beta = tf.Variable(np.random.normal(size=(DICT_SIZE, EMBED_SIZE))/np.sqrt(EMBED_SIZE), dtype=tf.float32)
    tf_nce_intercept = tf.Variable(np.zeros(shape=(DICT_SIZE)), dtype=tf.float32)
    tf_nce_loss = tf.reduce_mean(
                    tf.nn.nce_loss(weights=tf_nce_beta, biases=tf_nce_intercept,
                                   labels=tf_in_context, inputs=tf_embedded_word,
                                   num_sampled=NCE_NUM_SAMPLED, num_classes=DICT_SIZE))
    tf_train = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(tf_nce_loss)
    
with tf.name_scope('Validation'):
    tf_valid_dictionary = tf.constant(np.array(range(DICT_SIZE)))
    tf_valid_embedding = tf.nn.embedding_lookup(tf_embedding, tf_valid_dictionary)
    tf_valid_in_norm = tf_embedded_word / tf.sqrt(tf.reduce_sum(tf.square(tf_embedded_word), 1, keep_dims=True))
    tf_valid_dic_norm = tf_valid_embedding / tf.sqrt(tf.reduce_sum(tf.square(tf_valid_embedding), 1, keep_dims=True))
    tf_valid_similarity = tf.matmul(tf_valid_in_norm, tf_valid_dic_norm, transpose_b=True)
    
print('Graph creation complete.')

Graph creation complete.


In [4]:
test_input = np.array([2, 32])
tf_saver = tf.train.Saver()

with tf.Session() as tfs:
    tf_saver.restore(tfs, save_path='../mltest/')
    res = tf_embedding.eval(feed_dict={tf_in_word: test_input})

INFO:tensorflow:Restoring parameters from ../mltest/


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ../mltest/
	 [[Node: save/RestoreV2_7 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save/Const_0_0, save/RestoreV2_7/tensor_names, save/RestoreV2_7/shape_and_slices)]]

Caused by op 'save/RestoreV2_7', defined at:
  File "C:\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start(

In [5]:
res

NameError: name 'res' is not defined

In [4]:
#Load state

with open(stat_file, 'rb') as f:
    statNwords = pickle.load(f)

with open(w2vsrc_file, 'rb') as f:
    w2v_src = pickle.load(f)
    
mapper = {x[0]:i for (i,x) in enumerate(statNwords)}

def word2idx(w):
    if w in mapper:
        return mapper[w]+1
    else:
        return 0
    
def idx2word(i):
    if i == 0:
        return '<unk>'
    if i-1 >= len(statNwords):
        return '<err>'
    return statNwords[i-1][0]